In [8]:
# Import libraries
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set plotting settings
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=100, frameon=False)

BASE_DIR = "/beegfs/scratch/ric.broccoli/kubacki.michal/SRF_Linda_RNA"

os.chdir(BASE_DIR)

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
samples = {
    "Emx1_Ctrl": "cellranger_counts_R26_Emx1_Ctrl_adult_0",
    "Emx1_Mut": "cellranger_counts_R26_Emx1_Mut_adult_1",
    "Nestin_Ctrl": "cellranger_counts_R26_Nestin_Ctrl_adult_2",
    "Nestin_Mut": "cellranger_counts_R26_Nestin_Mut_adult_3"
    }

# 1. Setup and Data Loading

In [13]:
# Path to the matrix files
CELL_DATA_DIR = "cellranger_final_count_data"

SAMPLE = samples["Emx1_Mut"]
matrix_dir = os.path.join(BASE_DIR, CELL_DATA_DIR, SAMPLE, "outs", "filtered_feature_bc_matrix")

# Load the data from the filtered matrix
try:
    adata = sc.read_10x_mtx(
        matrix_dir,
        var_names='gene_symbols',
        cache=True
    )
    print(f"Shape of loaded data: {adata.shape}")  # cells × genes
except ValueError as e:
    print(f"Error loading data: {e}")
    # Try loading with different parameters to handle the mismatch
    adata = sc.read_10x_mtx(
        matrix_dir,
        var_names='gene_symbols',
        cache=False
    )
    print(f"Shape of loaded data after retry: {adata.shape}")  # cells × genes

# 2. Basic Pre-processing

In [14]:
# Make a copy of the raw counts
adata.raw = adata.copy()

# Basic filtering
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# Calculate QC metrics
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # identify mitochondrial genes
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# Plot QC metrics
fig, axs = plt.subplots(1, 3, figsize=(15, 4))
sns.histplot(adata.obs['n_genes_by_counts'], kde=False, ax=axs[0])
axs[0].set_title('Genes per cell')
sns.histplot(adata.obs['total_counts'], kde=False, ax=axs[1])
axs[1].set_title('UMI counts per cell')
sns.histplot(adata.obs['pct_counts_mt'], kde=False, ax=axs[2])
axs[2].set_title('Percent mitochondrial')
plt.tight_layout()
plt.show()

# 3. Filtering Based on QC Metrics

In [15]:
max_genes = 15000 
min_genes = 500  
max_mt_pct = 20  

adata = adata[adata.obs['n_genes_by_counts'] < max_genes, :]
adata = adata[adata.obs['n_genes_by_counts'] > min_genes, :]
adata = adata[adata.obs['pct_counts_mt'] < max_mt_pct, :]

print(f"Number of cells after filtering: {adata.n_obs}")
print(f"Number of genes after filtering: {adata.n_vars}")

# 4. Normalization and Log Transformation

In [16]:
# Normalize to 10,000 reads per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# Log transform
sc.pp.log1p(adata)

# Identify highly variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
print(f"Number of highly variable genes: {sum(adata.var.highly_variable)}")

# Plot highly variable genes
plt.figure(figsize=(10, 8))
sc.pl.highly_variable_genes(adata, show=False)
plt.tight_layout()
plt.show()

# Keep only highly variable genes for dimensionality reduction
adata_hvg = adata[:, adata.var.highly_variable]

# 5. Dimensionality Reduction

In [17]:
# Scale data to unit variance and zero mean
sc.pp.scale(adata_hvg, max_value=10)

# Run PCA
sc.tl.pca(adata_hvg, svd_solver='arpack')

# Determine number of significant PCs
sc.pl.pca_variance_ratio(adata_hvg, n_pcs=50, log=True)
plt.show()

# Choose number of PCs for downstream analyses
n_pcs = 30  # Adjust based on the variance ratio plot

# Compute neighborhood graph
sc.pp.neighbors(adata_hvg, n_neighbors=15, n_pcs=n_pcs)

# Run UMAP
sc.tl.umap(adata_hvg)

# Plot UMAP
plt.figure(figsize=(10, 8))
sc.pl.umap(adata_hvg, color=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
           use_raw=False, color_map='viridis', show=False)
plt.tight_layout()
plt.show()

# 6. Clustering

In [18]:
# # Find clusters using Leiden algorithm
# sc.tl.leiden(adata_hvg, resolution=0.5)  # Adjust resolution as needed

# # Plot clusters on UMAP
# plt.figure(figsize=(10, 8))
# sc.pl.umap(adata_hvg, color='leiden', show=False)
# plt.tight_layout()
# plt.show()

# # Check cluster sizes
# cluster_counts = adata_hvg.obs['leiden'].value_counts()
# print(cluster_counts)

In [19]:
# Try different resolutions to find optimal number of clusters
resolutions = [0.1, 0.3, 0.5, 0.7, 1.0]
for res in resolutions:
    sc.tl.leiden(adata_hvg, resolution=res, key_added=f'leiden_res{res}')

# Plot clusters at different resolutions with improved layout
fig, axes = plt.subplots(1, len(resolutions), figsize=(20, 5))
for i, res in enumerate(resolutions):
    sc.pl.umap(adata_hvg, color=f'leiden_res{res}', title=f'Resolution {res}', 
               frameon=True, legend_loc='on data', legend_fontsize=10, ax=axes[i], show=False)

# Ensure proper spacing between subplots
plt.tight_layout()
plt.show()

# 7. Marker Gene Identification

In [20]:
print(adata_hvg.layers.keys())
print(adata_hvg)

In [21]:
from scipy.sparse import issparse
import numpy as np

# Check first 5 values from first cell
if issparse(adata_hvg.X):
    print("X matrix values (first cell):", adata_hvg.X[0, :5].toarray().flatten())
else:
    print("X matrix values (first cell):", adata_hvg.X[0, :5])
print("Should be log1p transformed values (~0-5 range)")

# Check raw values if raw exists
if adata_hvg.raw:
    if issparse(adata_hvg.raw.X):
        print("Raw values:", adata_hvg.raw.X[0, :5].toarray().flatten())
    else:
        print("Raw values:", adata_hvg.raw.X[0, :5])
    print("Should be original counts (integers)")

In [23]:
leiden_resolution = "leiden_res0.1"

In [24]:
sc.tl.rank_genes_groups(adata_hvg, leiden_resolution, method='wilcoxon', use_raw=False)

# Plot top marker genes
plt.figure(figsize=(15, 10))
sc.pl.rank_genes_groups(adata_hvg, n_genes=15, sharey=False, show=False)
plt.tight_layout()
plt.show()

# Get the top markers for each cluster
marker_genes = pd.DataFrame()
for i in range(len(np.unique(adata_hvg.obs[leiden_resolution]))):
    markers = sc.get.rank_genes_groups_df(adata_hvg, group=str(i))
    markers = markers.sort_values('pvals_adj')
    markers['cluster'] = i
    marker_genes = pd.concat([marker_genes, markers.head(10)])

# Save markers to CSV
marker_genes.to_csv('cluster_markers.csv', index=False)

# Heatmap of top markers per cluster
top_markers_per_cluster = {}
for cluster in np.unique(adata_hvg.obs[leiden_resolution]):
    cluster_markers = marker_genes[marker_genes['cluster'] == int(cluster)]
    top_markers_per_cluster[cluster] = cluster_markers['names'].tolist()[:5]

markers_flat = [gene for cluster_markers in top_markers_per_cluster.values() for gene in cluster_markers]
markers_unique = list(dict.fromkeys(markers_flat))  # Remove duplicates while preserving order

# Run dendrogram to avoid warning
sc.tl.dendrogram(adata_hvg, groupby=leiden_resolution)

plt.figure(figsize=(15, 10))

try:
    sc.pl.heatmap(adata_hvg, markers_unique, groupby=leiden_resolution, dendrogram=True, 
                  swap_axes=True, use_raw=False, show=False, standard_scale='var')
    plt.tight_layout()
    plt.show()
except KeyError as e:
    print(f"KeyError encountered: {e}")
    print("Check if the marker genes are in adata_hvg.var_names.")
    
    # Find missing genes
    missing_genes = [gene for gene in markers_unique if gene not in adata_hvg.var_names]
    print(f"Missing genes: {missing_genes}")
    
    # Proceed with available genes, if any
    available_markers = [gene for gene in markers_unique if gene in adata_hvg.var_names]
    if available_markers:
        print("Plotting heatmap with available markers.")
        sc.pl.heatmap(adata_hvg, available_markers, groupby=leiden_resolution, dendrogram=True, 
                      swap_axes=True, use_raw=False, show=False, standard_scale='var')
        plt.tight_layout()
        plt.show()
    else:
        print("No valid marker genes found in adata_hvg.var_names. Heatmap cannot be plotted.")

# 8. Cell Type Annotation

In [26]:
def annotate_cell_types(adata):
    cell_type_markers = {
        'Neurons': ['Rbfox3', 'Tubb3', 'Map2', 'Snap25'],
        'Astrocytes': ['Gfap', 'Aqp4', 'Aldh1l1'],
        'Oligodendrocytes': ['Mbp', 'Plp1', 'Mog', 'Olig1', 'Olig2'],
        'Microglia': ['Cx3cr1', 'P2ry12', 'Hexb', 'Csf1r'],
        'OPCs': ['Pdgfra', 'Cspg4'],
        'Endothelial': ['Cldn5', 'Pecam1', 'Vwf'],
        'Pericytes': ['Pdgfrb', 'Rgs5', 'Acta2']
    }
    
    # Create a new dataframe to store scores
    cell_type_scores = pd.DataFrame(index=adata.obs_names)
    
    # For each cell type, calculate the mean expression of marker genes
    for cell_type, markers in cell_type_markers.items():
        # Find markers that exist in the dataset
        existing_markers = [marker for marker in markers if marker in adata.var_names]
        if len(existing_markers) > 0:
            # Calculate mean expression of markers for each cell
            cell_type_scores[cell_type] = adata[:, existing_markers].X.mean(axis=1)
    
    # Assign cell type based on highest score
    adata.obs['cell_type'] = cell_type_scores.idxmax(axis=1)
    
    # Add score for visualization
    adata.obs['cell_type_score'] = cell_type_scores.max(axis=1)
    
    return adata

# Annotate cell types
adata_hvg = annotate_cell_types(adata_hvg)

# Plot cell types on UMAP
plt.figure(figsize=(8, 6))
sc.pl.umap(adata_hvg, color='cell_type', show=False)
plt.tight_layout()
plt.show()

# Compare with Leiden clusters
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sc.pl.umap(adata_hvg, color=leiden_resolution, title='Leiden clusters', ax=axes[0], show=False)
sc.pl.umap(adata_hvg, color='cell_type', title='Cell types', ax=axes[1], show=False)
plt.tight_layout()
plt.show()

# 9. Differential Expression Analysis

In [27]:
selected_cluster = '6'

# Perform differential expression between all clusters vs. the rest
sc.tl.rank_genes_groups(adata_hvg, leiden_resolution, method='wilcoxon', use_raw=False)

# Plot results - violin plot for selected cluster
plt.figure(figsize=(10, 8))
try:
    sc.pl.rank_genes_groups_violin(adata_hvg, groups=[selected_cluster], n_genes=10, show=False)
    plt.tight_layout()
    plt.show()
except ValueError as e:
    print(f"Error plotting violin plot: {e}. Skipping violin plot.")


# Plot results - rank genes groups plot
try:
    sc.pl.rank_genes_groups(adata_hvg, n_genes=25, sharey=False)
    plt.tight_layout()
    plt.show()
except ValueError as e:
    print(f"Error plotting rank genes groups plot: {e}. Skipping rank genes groups plot.")


# Get differential expression results as dataframe for selected cluster
de_genes = sc.get.rank_genes_groups_df(adata_hvg, group=selected_cluster, key='rank_genes_groups')
print(de_genes.head(20))

# 10. Save Results

In [29]:
OUT_DIR = "post_analysis/results"
os.makedirs(os.path.join(OUT_DIR, SAMPLE), exist_ok=True)

# Save the processed object for future use
adata_hvg.write(os.path.join(OUT_DIR, SAMPLE, 'R26_Nestin_Mut_adult_processed.h5ad'))

# Save key results to CSV files
adata_hvg.obs.to_csv(os.path.join(OUT_DIR, SAMPLE, 'cell_metadata.c     sv'))
pd.DataFrame({'UMAP1': adata_hvg.obsm['X_umap'][:, 0], 
              'UMAP2': adata_hvg.obsm['X_umap'][:, 1], 
              'cluster': adata_hvg.obs[leiden_resolution],
              'cell_type': adata_hvg.obs['cell_type']}).to_csv(os.path.join(OUT_DIR, SAMPLE, 'umap_coordinates.csv'), index=False)

# Generate summary report
summary = {
    'total_cells': adata_hvg.n_obs,
    'total_genes': adata_hvg.n_vars,
    'clusters': len(np.unique(adata_hvg.obs[leiden_resolution])),
    'cell_types': len(np.unique(adata_hvg.obs['cell_type'])),
    'cells_per_cluster': adata_hvg.obs[leiden_resolution].value_counts().to_dict(),
    'cells_per_cell_type': adata_hvg.obs['cell_type'].value_counts().to_dict()
}

with open(os.path.join(OUT_DIR, SAMPLE, 'analysis_summary.txt'), 'w') as f:
    for key, value in summary.items():
        f.write(f"{key}: {value}\n")